# **Bioinformatics Project - Computational Drug Discovery [Part 2] 

Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)

---

## **Install conda and rdkit**

In [1]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-01-09 10:35:50--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   157MB/s    in 0.5s    

2022-01-09 10:35:51 (157 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

## **Load bioactivity data**

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/sars-cov-2_bioactivity_data_preprocessed.csv')

In [8]:
df

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL3301610,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,6620.0,intermediate
1,CHEMBL682,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,5150.0,intermediate
2,CHEMBL264241,CCCCCOc1ccc(-c2ccc(-c3ccc(C(=O)N[C@H]4C[C@@H](...,4640.0,intermediate
3,CHEMBL46740,Cc1c(-c2ccc(O)cc2)n(Cc2ccc(OCCN3CCCCCC3)cc2)c2...,3440.0,intermediate
4,CHEMBL504323,COc1cc2c3cc1Oc1c(OC)c(OC)cc4c1[C@@H](Cc1ccc(O)...,7870.0,intermediate
...,...,...,...,...
281,CHEMBL583194,c1cncc(CN2CCC(n3ncc4c(N5CCOCC5)nc(-c5ccc6[nH]c...,9430.0,intermediate
282,CHEMBL3967196,CCCCn1c(NC(=O)c2cccs2)c(C#N)c2nc3ccccc3nc21,19650.0,inactive
283,CHEMBL601661,CNC(=O)Nc1ccc(-c2nc(N3CC4CCC(C3)O4)c3cnn(C4CCC...,21620.0,inactive
284,CHEMBL178334,CC(C)N1CCN(c2ccc(C(=O)c3c(-c4ccc(O)cc4)sc4cc(O...,29870.0,inactive


## **Calculate Lipinski descriptors**
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the **druglikeness** of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the **Rule-of-Five** or **Lipinski's Rule**.

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10 

### **Import libraries**

In [5]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

### **Convert IC50 to pIC50**
To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

This custom function pIC50() will accept a DataFrame as input and will:
* Take the IC50 values from the ``standard_value`` column and converts it from nM to M by multiplying the value by 10$^{-9}$
* Take the molar value and apply -log10
* Delete the ``standard_value`` column and create a new ``pIC50`` column

In [9]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [10]:
df.standard_value.describe()

count      286.000000
mean      8260.443717
std       9395.814899
min          6.653000
25%       2086.990000
50%       5365.160000
75%      11735.000000
max      91201.080000
Name: standard_value, dtype: float64

In [11]:
-np.log10( (10**-9)* 100000000 )

1.0

In [12]:
-np.log10( (10**-9)* 10000000000 )

-1.0

In [13]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [14]:
df_norm = norm_value(df)
df_norm

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value_norm
0,CHEMBL3301610,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,intermediate,6620.0
1,CHEMBL682,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,intermediate,5150.0
2,CHEMBL264241,CCCCCOc1ccc(-c2ccc(-c3ccc(C(=O)N[C@H]4C[C@@H](...,intermediate,4640.0
3,CHEMBL46740,Cc1c(-c2ccc(O)cc2)n(Cc2ccc(OCCN3CCCCCC3)cc2)c2...,intermediate,3440.0
4,CHEMBL504323,COc1cc2c3cc1Oc1c(OC)c(OC)cc4c1[C@@H](Cc1ccc(O)...,intermediate,7870.0
...,...,...,...,...
281,CHEMBL583194,c1cncc(CN2CCC(n3ncc4c(N5CCOCC5)nc(-c5ccc6[nH]c...,intermediate,9430.0
282,CHEMBL3967196,CCCCn1c(NC(=O)c2cccs2)c(C#N)c2nc3ccccc3nc21,inactive,19650.0
283,CHEMBL601661,CNC(=O)Nc1ccc(-c2nc(N3CC4CCC(C3)O4)c3cnn(C4CCC...,inactive,21620.0
284,CHEMBL178334,CC(C)N1CCN(c2ccc(C(=O)c3c(-c4ccc(O)cc4)sc4cc(O...,inactive,29870.0


In [15]:
df_norm.standard_value_norm.describe()

count      286.000000
mean      8260.443717
std       9395.814899
min          6.653000
25%       2086.990000
50%       5365.160000
75%      11735.000000
max      91201.080000
Name: standard_value_norm, dtype: float64

In [16]:
df_final = pIC50(df_norm)
df_final

,molecule_chembl_id,canonical_smiles,bioactivity_class,pIC50
0,CHEMBL3301610,CCN1CCN(Cc2ccc(Nc3ncc(F)c(-c4cc(F)c5nc(C)n(C(C...,intermediate,5.179142
1,CHEMBL682,CCN(CC)Cc1cc(Nc2ccnc3cc(Cl)ccc23)ccc1O,intermediate,5.288193
2,CHEMBL264241,CCCCCOc1ccc(-c2ccc(-c3ccc(C(=O)N[C@H]4C[C@@H](...,intermediate,5.333482
3,CHEMBL46740,Cc1c(-c2ccc(O)cc2)n(Cc2ccc(OCCN3CCCCCC3)cc2)c2...,intermediate,5.463442
4,CHEMBL504323,COc1cc2c3cc1Oc1c(OC)c(OC)cc4c1[C@@H](Cc1ccc(O)...,intermediate,5.104025
...,...,...,...,...
281,CHEMBL583194,c1cncc(CN2CCC(n3ncc4c(N5CCOCC5)nc(-c5ccc6[nH]c...,intermediate,5.025488
282,CHEMBL3967196,CCCCn1c(NC(=O)c2cccs2)c(C#N)c2nc3ccccc3nc21,inactive,4.706637
283,CHEMBL601661,CNC(=O)Nc1ccc(-c2nc(N3CC4CCC(C3)O4)c3cnn(C4CCC...,inactive,4.665144
284,CHEMBL178334,CC(C)N1CCN(c2ccc(C(=O)c3c(-c4ccc(O)cc4)sc4cc(O...,inactive,4.524765


In [17]:
df_final.pIC50.describe()

count    286.000000
mean       5.402166
std        0.663532
min        4.040000
25%        4.930520
50%        5.270418
75%        5.680522
max        8.176982
Name: pIC50, dtype: float64

### **Removing the 'intermediate' bioactivity class**
Here, we will be removing the ``intermediate`` class from our data set.

In [18]:
df_2class = df_final[df_final.bioactivity_class != 'intermediate']
df_2class

,molecule_chembl_id,canonical_smiles,bioactivity_class,pIC50
10,CHEMBL1751,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,active,6.721246
11,CHEMBL1526240,CC(C)=CCCC1(C)C=Cc2c(O)c3c(c(CC=C(C)C)c2O1)OC1...,inactive,4.873869
17,CHEMBL496,Oc1c(Cl)cc(Cl)c(Cl)c1Cc1c(O)c(Cl)cc(Cl)c1Cl,active,6.045757
28,CHEMBL1448,O=C(Nc1ccc([N+](=O)[O-])cc1Cl)c1cc(Cl)ccc1O,active,6.552842
34,CHEMBL1242,Nc1ccc(/N=N/c2ccccc2)c(N)n1,inactive,4.552842
...,...,...,...,...
278,CHEMBL3913746,CC(C)(O)CNc1ncc(-c2ccnc(Nc3ccnc(Cl)c3)n2)c(CC2...,active,6.397940
279,CHEMBL1254577,O=C(NCCN1CCC2(CC1)C(=O)NCN2c1cccc(F)c1)c1ccc2c...,inactive,4.934047
282,CHEMBL3967196,CCCCn1c(NC(=O)c2cccs2)c(C#N)c2nc3ccccc3nc21,inactive,4.706637
283,CHEMBL601661,CNC(=O)Nc1ccc(-c2nc(N3CC4CCC(C3)O4)c3cnn(C4CCC...,inactive,4.665144


In [19]:
df_2class.to_csv('sars-cov-2_bioactivity_data_processed_2classes_pIC50.csv')

---